# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.data.dataset_factory import FileDatasetFactory
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from main import register_image_data_as_table

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'covid-pneumonia-automl'

experiment=Experiment(ws, experiment_name)

compute_name = "compute-cluster"
vm_size = "STANDARD_NC6"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print("Found existing cluster, using it!")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=6)
    compute_cluster = ComputeTarget.create(ws, compute_name, compute_config)
compute_cluster.wait_for_completion()

In [ ]:
register_image_data_as_table(ws, './Covid19-dataset/train', "covid-pneumonia-dataset-table")

data = Dataset.get_by_name(workspace=ws, name="covid-pneumonia-dataset-table")
## try to run this command without registering the dataset and what error comes. Use that error as try except block and call register_data in the except block. Also do the same try except block in hyperdrive code

In [ ]:
df = data.to_pandas_dataframe()
df.head()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {"enable_dnn":True,
                   "experiment_timeout_minutes":60,
                   "max_cores_per_iteration":-1,
                   "enable_tf":False,
                   "featurization":"off"}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task='classification',
                             primary_metric='accuracy',
                             compute_target=compute_cluster,
                             training_data=data,
                             label_column_name='class',
                             **automl_settings
                             )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model
best_run = remote_run.get_best_child()
best_run_metrics = best_run.get_metrics()
best_run_parameters = best_run.get_details()
best_run_properties - best_run_parameters['properties']

print("Best Run:",best_run.id)
print("Algorithm:", best_run_properties['run_algorithm'])
print("Metric:", best_run_properties['primary_metric'])
print("Best Score:",best_run_properties['score'])


In [ ]:
best_run.get_file_names()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
registered_model = best_run.register_model(model_name='covid-pneumonia-automl',
                                            model_path='./outputs/model')


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service